In [2]:
import pandas as pd

In [3]:
data = pd.read_excel("Export System A September.xlsx")
print(data)

        CLIENT_MAN_ID Физ./юр. лицо    Резидент      Страна  \
0                3961     Физ. лицо    резидент   Казахстан   
1                4005     Физ. лицо    резидент   Казахстан   
2                4452     Физ. лицо    резидент   Казахстан   
3                9484     Физ. лицо    резидент   Казахстан   
4                9484     Физ. лицо    резидент   Казахстан   
...               ...           ...         ...         ...   
143354     9013659292     Физ. лицо  нерезидент      Грузия   
143355     9013659303     Физ. лицо  нерезидент  Узбекистан   
143356     9013659308     Физ. лицо  нерезидент      Россия   
143357     9013659333     Физ. лицо  нерезидент  Узбекистан   
143358     9010795737     Физ. лицо    резидент   Казахстан   

            Категория полис  \
0              новые полисы   
1              новые полисы   
2              новые полисы   
3       расторгнутые полисы   
4       расторгнутые полисы   
...                     ...   
143354         новые полис

In [4]:
clients = len(data["CLIENT_MAN_ID"])
unique_clients = len(set(data["CLIENT_MAN_ID"]))
if clients != unique_clients:
    print(f"Клиенты повторяются в списке. Кол-во уникальных клиентов: {unique_clients}")
else:
    print(f"Клиенты не повторяются в списке. Их кол-во: {unique_clients}")

Клиенты повторяются в списке. Кол-во уникальных клиентов: 100595


In [5]:
legal_entity_data = data[data["Физ./юр. лицо"] == "Юр. лицо"]
person_data = data[data["Физ./юр. лицо"] == "Физ. лицо"]

legal_entity_num = len(set(legal_entity_data["CLIENT_MAN_ID"]))
person_num = len(set(person_data["CLIENT_MAN_ID"]))

if legal_entity_num + person_num == unique_clients:
    print(f"Данные верны. В сумме кол-во юр. лиц и физ. лиц равны общему кол-ву клиентов\n{legal_entity_num} + {person_num} = {unique_clients}")
else:
    print("Данные неверны.")

Данные верны. В сумме кол-во юр. лиц и физ. лиц равны общему кол-ву клиентов
1997 + 98598 = 100595


In [18]:
policy_status = data.groupby("статус полиса").size()
additional_status = data.groupby("доп. статус").size()
print(policy_status.to_string(), '\n-------------------\n', additional_status.to_string())

статус полиса
аннулирован           3621
завершенный полис    32638
зарегистр. полис     50594
оплаченный полис     56506 
-------------------
 доп. статус
Аннулирован      6053
Нормальный     107102
Расторгнут      30204


In [29]:
data['Дат.выдачи пол'] = pd.to_datetime(data['Дат.выдачи пол'])
filtered_data_part1 = data[(data['Дат.выдачи пол'].dt.month == 9) & 
                    data['Дат.выдачи пол'].dt.day.between(1, 15) & 
                    (data['доп. статус'] == 'Нормальный')]
print("Result:", len(filtered_data_part1))

Result: 54280


In [28]:
data['Дат.выдачи пол'] = pd.to_datetime(data['Дат.выдачи пол'])
filtered_data_part2 = data[(data['Дат.выдачи пол'].dt.month == 9) & 
                    data['Дат.выдачи пол'].dt.day.between(15, 30) & 
                    (data['доп. статус'] == 'Нормальный')]
print("Result:", len(filtered_data_part2))

Result: 51775


In [30]:
print(f"Премиальные за первую половину сентября: {filtered_data_part1['Премия по дог-ру'].sum()}")
print(f"Премиальные за вторую половину сентября: {filtered_data_part2['Премия по дог-ру'].sum()}")

Премиальные за первую половину сентября: 996478719.1600001
Премиальные за вторую половину сентября: 1014190235.3


In [38]:
data2 = pd.read_excel("Report_01092021-15092021.xls", sheet_name="Статистика ввода")
data3 = pd.read_excel("Report_16092021-30092021.xls", sheet_name="Статистика ввода")

merged_data2 = pd.merge(filtered_data_part1, data2, left_on="policy_id", right_on="№ полиса")
merged_data3= pd.merge(filtered_data_part2, data3, left_on="policy_id", right_on="№ полиса")

In [42]:
september_1_15 = merged_data2[merged_data2['Премия по дог-ру'] != merged_data2['Рассчитанная премия']]
september_15_30 = merged_data3[merged_data3['Премия по дог-ру'] != merged_data3['Рассчитанная премия']]

print(f"Кол-во полисов, не совпадающих за первую половину сентября: {len(september_1_15)}.")
print(f"Кол-во полисов, не совпадающих за вторую половину сентября: {len(september_15_30)}.")

Кол-во полисов, не совпадающих за первую половину сентября: 8687.
Кол-во полисов, не совпадающих за вторую половину сентября: 7192.


In [57]:
import os

def createFile(x, filename):
    filepath = os.getcwd()

    if not (os.path.exists(f"{filepath}/Non-matching policies{filename}.xlsx")):
        x.to_excel(f'Non-matching policies{filename}.xlsx', sheet_name='Sheet1', index=False)
        print(f"Excel файл с дополнительной информацией создан успешно. Файл находится по пути {filepath}")
    else:
        print(f'Файл по пути {filepath} уже существует. Выберите другое имя или путь.')

In [58]:
createFile(september_1_15, "[September1-15]")
createFile(september_15_30, "[September_15-30]")

Excel файл с дополнительной информацией создан успешно. Файл находится по пути c:\Users\user\Desktop\Homework for candidates
Excel файл с дополнительной информацией создан успешно. Файл находится по пути c:\Users\user\Desktop\Homework for candidates
